##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pérdidas de componentes de TensorFlow: TripletSemiHardLoss

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/losses_triplet"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/losses_triplet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/addons/tutorials/losses_triplet.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/addons/tutorials/losses_triplet.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

## Descripción general

En estas notas se demuestra cómo usar la función TripletSemiHardLoss en complementos de TensorFlow.

### Recursos:

- [FaceNet:  A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf) (FaceNet: incorporaciones unificadas para reconocimiento facial y agrupación en clústeres)
- [En el blog de Oliver Moindrot hay un trabajo excelente en el que se describe en detalle el algoritmo.](https://omoindrot.github.io/triplet-loss)


## TripletLoss

Tal como se presenta en la publicación sobre FaceNet, TripletLoss (la pérdida triplete) es una función de pérdida que entrena a una red neuronal para que incorpore cerca características de la misma clase y que, a la vez, maximice la distancia entre las incorporaciones de diferentes clases. Para lograrlo, se elige un anclaje junto con una muestra negativa y una positiva. ![Figura 3](https://user-images.githubusercontent.com/18154355/61485418-1cbb1f00-a96f-11e9-8de8-3c46eef5a7dc.png)

**La función de pérdida se describe como función de distancia euclidiana:**

![Función](https://user-images.githubusercontent.com/18154355/61484709-7589b800-a96d-11e9-9c3c-e880514af4b7.png)

Donde A es nuestra entrada de anclaje (anchor), P es la entrada de muestra positiva, N es la entrada de muestra negativa y alfa es algo cierto que se usa para especificar cuándo un triplete se ha vuelto demasiado "fácil" y ya no se desea ajustar los pesos con él.

## Aprendizaje en línea SemiHard

Tal como se muestra en la publicación, los mejores resultados se obtienen de tripletes conocidos como "semiduros". Son aquellos que se los define como tripletes en los que el negativo está más lejos del anclaje que el positivo, pero que aún producen una pérdida positiva. Para hallar estos tripletes con eficiencia, se usa el aprendizaje en línea y solamente se entrena a partir de los ejemplos semiduros en cada lote.


## Preparación

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import io
import numpy as np

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

## Preparación de los datos

In [ ]:
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)

train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

# Build your input pipelines
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.map(_normalize_img)

test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(_normalize_img)

## Generación del modelo

![Figura 2](https://user-images.githubusercontent.com/18154355/61485417-1cbb1f00-a96f-11e9-8d6a-94964ce8c4db.png)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

## Entrenamiento y evaluación

In [ ]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [ ]:
# Train the network
history = model.fit(
    train_dataset,
    epochs=5)

In [ ]:
# Evaluate the network
results = model.predict(test_dataset)

In [ ]:
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

## Proyector de incorporación

Los archivos de metadatos y del vector se pueden cargar y visualizar aquí: https://projector.tensorflow.org/

Los resultados de nuestra prueba de incorporación (<em>embedding</em>) se pueden ver con UMAP: ![Incorporación](https://user-images.githubusercontent.com/18154355/61600295-e6470380-abfd-11e9-8a00-2b25e7e6916f.png)
